## xgboost tuning

In [2]:
import os
import sys
import scipy
import scipy.sparse
import pandas as pd
import numpy as np
import xgboost as xgb
import time
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV

PROJECT_DIR = '/home/michal/diplomka/code'
if PROJECT_DIR not in sys.path:
    sys.path.append(PROJECT_DIR)
from lib.utils import load_csr_matrix

/home/michal/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/michal/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
FEATURES = ['a_hypernyms', 'a_head_form', 'a_head_number', 'a_non_article_det', 'a_parent', 'a_pos_after_head', 'a_pos_before_head', 'a_words_after_head', 'a_words_after_np', 'a_words_before_head', 'a_words_before_np', 'b_head_proper', 'b_head_pos_simple', 'b_object_form', 'b_pos_after_head_as_list', 'b_pos_before_head_as_list', 'b_pp_object_form', 'b_postmodification_type', 'b_referent', 'b_words_after_head_as_list', 'b_words_after_np_as_list', 'b_words_before_head_as_list', 'b_words_before_np_as_list', 'c_countability_bnc', 'd_head_form_embeddings', 'e_kenlm_ggl_5_lc_nbs']
DATA_PATH = os.path.join(PROJECT_DIR, '../data/features/penn/postprocessed')
CV_RESULTS_LOG_PATH = os.path.join(PROJECT_DIR, 'logs/experiments/model_results/penn/xgboost')  # logovani vysledku z cross-validace
NUM_CLASS = 3
TRAIN_SET_NAME = 'train'
TEST_SET_NAME = 'heldout'

### Načtení dat:

In [4]:
def load_data(features, dataset_name, sample=None):
    y = np.load(os.path.join(DATA_PATH, dataset_name, 'Y_article'))
    if sample:
        np.random.seed(seed=42)
        sample_indices = [np.random.choice(len(y), sample, replace=False)]
        y = y[sample_indices]
        
    feature_matrices = []
    for feature_name in features:
        feature_matrix = load_csr_matrix(
            os.path.join(DATA_PATH, dataset_name, feature_name + '.npz')
        )
        if sample:
            feature_matrix = scipy.sparse.csr_matrix(feature_matrix.toarray()[sample_indices])
        feature_matrices.append(feature_matrix)
    x =  scipy.sparse.hstack(feature_matrices)
    assert x.shape[0] == len(y)
    return x, y

In [5]:
train_x, train_y = load_data(FEATURES, TRAIN_SET_NAME)
test_x, test_y = load_data(FEATURES, TEST_SET_NAME)
assert train_x.shape[1] == test_x.shape[1]
print(train_x.shape, test_x.shape)

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train_y)
train_y = le.transform(train_y)
test_y = le.transform(test_y)

(263088, 287762) (10076, 287762)


In [13]:
xgtrain = xgb.DMatrix(train_x, label=train_y)

### Ladění

In [12]:
XGB_DEFAULT_PARAMS = { 
    'learning_rate': 0.1,
    'n_estimators': 2000,
    'max_depth': 5,
    'min_child_weight': 1,
    'gamma': 0,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'objective': 'multi:softmax',
    'nthread': 24,
    'scale_pos_weight': 1,
    'seed': 27
}

#### počet stromů vs velikost dat 

In [7]:
def cross_validate(model, train_x_dmatrix, cv_folds=5, early_stopping_rounds=20, log_file_name=None):
    start_time = time.time()
    xgb_param = model.get_xgb_params()
    xgb_param['num_class'] = NUM_CLASS
    cvresult = xgb.cv(
        xgb_param,
        train_x_dmatrix,
        num_boost_round=model.get_params()['n_estimators'],
        nfold=cv_folds,
        early_stopping_rounds=early_stopping_rounds,
        verbose_eval=True,
    )
    if log_file_name:
        cvresult.to_csv(os.path.join(CV_RESULTS_LOG_PATH, log_file_name))
    end_time = time.time()
    print("Done in {} minutes".format((end_time - start_time)/60))

In [8]:
def train_and_eval(model, train_x, train_y, test_x, test_y, log_file_name):
    start_time = time.time()
    model.fit(train_x, train_y)
    train_predictions = model.predict(train_x)
    test_predictions = model.predict(test_x)
    with open(os.path.join(CV_RESULTS_LOG_PATH, log_file_name), 'a+') as f:
        f.write(str(model.get_params()) + '\n')
        f.write("Accuracy ({}): {:.4f}\n".format(TRAIN_SET_NAME, metrics.accuracy_score(train_y, train_predictions)))
        f.write("Accuracy ({}): {:.4f}\n".format(TEST_SET_NAME, metrics.accuracy_score(test_y, test_predictions)))
    end_time = time.time()
    print("Done in {} minutes".format((end_time - start_time)/60))

10 000 trenovacich dat

In [56]:
train_x_10000, train_y_10000 = load_data(FEATURES, TRAIN_SET_NAME, sample=10000)
#xgtrain_10000 = xgb.DMatrix(train_x_10000, label=le.transform(train_y_10000))

In [ ]:
xgb2 = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, nthread=2))
cross_validate(xgb2, xgtrain_10000, log_file_name='n_estimators_010_000.csv')

In [78]:
xgb2_nestimators = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, nthread=-1, n_estimators=122))
train_and_eval(xgb2_nestimators, train_x_10000, le.transform(train_y_10000), test_x, test_y, log_file_name='010_000.txt')

20 000 trenovacich dat

In [54]:
train_x_20000, train_y_20000 = load_data(FEATURES, TRAIN_SET_NAME, sample=20000)
xgtrain_20000 = xgb.DMatrix(train_x_20000, label=le.transform(train_y_20000))

In [126]:
xgb2 = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, nthread=2))
cross_validate(xgb2, xgtrain_20000, log_file_name='n_estimators_020_000.csv')

[0]	train-merror:0.210688+0.00387246	test-merror:0.2181+0.00485438
[1]	train-merror:0.20105+0.0040184	test-merror:0.20955+0.00536284
[2]	train-merror:0.197788+0.00186738	test-merror:0.2072+0.00147817
[3]	train-merror:0.195712+0.00173227	test-merror:0.20315+0.00232702
[4]	train-merror:0.193225+0.00138437	test-merror:0.2004+0.00262964
[5]	train-merror:0.192712+0.00170981	test-merror:0.20025+0.00219659
[6]	train-merror:0.191425+0.0017038	test-merror:0.20005+0.00284341
[7]	train-merror:0.191037+0.00170594	test-merror:0.1998+0.00354048
[8]	train-merror:0.189375+0.00140888	test-merror:0.1984+0.00259615
[9]	train-merror:0.1877+0.00179903	test-merror:0.19765+0.00253279
[10]	train-merror:0.1874+0.002294	test-merror:0.19705+0.00280357
[11]	train-merror:0.186012+0.00257071	test-merror:0.19655+0.00255636
[12]	train-merror:0.184487+0.00203243	test-merror:0.19475+0.0033429
[13]	train-merror:0.182925+0.00204764	test-merror:0.1935+0.00267862
[14]	train-merror:0.181112+0.00271288	test-merror:0.1926+0.0

In [133]:
xgb2_nestimators = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, nthread=-1, n_estimators=271))
train_and_eval(xgb2_nestimators, train_x_20000, le.transform(train_y_20000), test_x, test_y, log_file_name='020_000.txt')

50 000 trenovacich dat

In [46]:
train_x_50000, train_y_50000 = load_data(FEATURES, TRAIN_SET_NAME, sample=50000)
#xgtrain_50000 = xgb.DMatrix(train_x_50000, label=le.transform(train_y_50000))

In [129]:
xgb3 = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, nthread=5))
cross_validate(xgb3, xgtrain_50000, log_file_name='n_estimators_050_000.csv')

[0]	train-merror:0.217305+0.0083856	test-merror:0.2196+0.0121631
[1]	train-merror:0.21054+0.00643499	test-merror:0.2139+0.0100584
[2]	train-merror:0.204+0.00250559	test-merror:0.2068+0.00438908
[3]	train-merror:0.19986+0.00288633	test-merror:0.2025+0.00501956
[4]	train-merror:0.198005+0.00202011	test-merror:0.20134+0.00463318
[5]	train-merror:0.196845+0.00129145	test-merror:0.20006+0.00388567
[6]	train-merror:0.19596+0.00116131	test-merror:0.19918+0.00334509
[7]	train-merror:0.19453+0.000958045	test-merror:0.19762+0.00194874
[8]	train-merror:0.19248+0.00112042	test-merror:0.19632+0.00179265
[9]	train-merror:0.19184+0.000665132	test-merror:0.19562+0.00256234
[10]	train-merror:0.19153+0.000605888	test-merror:0.195+0.00158619
[11]	train-merror:0.189815+0.000895209	test-merror:0.19356+0.00213972
[12]	train-merror:0.1879+0.00110329	test-merror:0.19164+0.00194792
[13]	train-merror:0.18735+0.00118279	test-merror:0.19112+0.00153805
[14]	train-merror:0.18549+0.00094652	test-merror:0.18934+0.002

In [142]:
xgb3_nestimators = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, nthread=-1, n_estimators=616))
train_and_eval(xgb3_nestimators, train_x_50000, le.transform(train_y_50000), test_x, test_y, log_file_name='050_000.txt')

100 000 trenovacich dat

In [60]:
train_x_100_000, train_y_100_000 = load_data(FEATURES, TRAIN_SET_NAME, sample=100000)
xgtrain_100_000 = xgb.DMatrix(train_x_100_000, label=le.transform(train_y_100_000))

In [133]:
xgb4 = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, nthread=10))
cross_validate(xgb4, xgtrain_100_000, log_file_name='n_estimators_100_000.csv')

[0]	train-merror:0.216305+0.00482675	test-merror:0.21863+0.00844456
[1]	train-merror:0.203745+0.00427971	test-merror:0.2056+0.00692221
[2]	train-merror:0.20378+0.00382951	test-merror:0.20524+0.00484308
[3]	train-merror:0.200988+0.0036742	test-merror:0.20152+0.00398693
[4]	train-merror:0.197585+0.00235783	test-merror:0.19856+0.00410212
[5]	train-merror:0.19741+0.00252084	test-merror:0.19884+0.00219121
[6]	train-merror:0.1962+0.00240151	test-merror:0.19765+0.00414258
[7]	train-merror:0.195385+0.00291249	test-merror:0.19678+0.00424424
[8]	train-merror:0.193585+0.00181182	test-merror:0.19486+0.00490473
[9]	train-merror:0.19239+0.00172155	test-merror:0.1936+0.00482068
[10]	train-merror:0.19128+0.00190788	test-merror:0.19271+0.00458611
[11]	train-merror:0.190508+0.00164712	test-merror:0.19244+0.00426713
[12]	train-merror:0.189338+0.000706628	test-merror:0.1911+0.00458268
[13]	train-merror:0.188552+0.000795318	test-merror:0.19008+0.00453692
[14]	train-merror:0.187425+0.000854418	test-merror:0

In [145]:
xgb_nestimators = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, nthread=-1, n_estimators=809))
train_and_eval(xgb_nestimators, train_x_100_000, le.transform(train_y_100_000), test_x, test_y, log_file_name='100_000.txt')

150 000 trenovacich dat

In [152]:
train_x_150_000, train_y_150_000 = load_data(FEATURES, TRAIN_SET_NAME, sample=150000)
xgtrain_150_000 = xgb.DMatrix(train_x_150_000, label=le.transform(train_y_150_000))

In [135]:
xgb5 = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, nthread=10))
cross_validate(xgb5, xgtrain_150_000, log_file_name='n_estimators_150_000.csv')

[0]	train-merror:0.222197+0.00964788	test-merror:0.222867+0.0105012
[1]	train-merror:0.208937+0.00683488	test-merror:0.20952+0.00714634
[2]	train-merror:0.20099+0.002238	test-merror:0.202033+0.00168648
[3]	train-merror:0.199275+0.00202217	test-merror:0.20006+0.00279004
[4]	train-merror:0.196817+0.00203066	test-merror:0.19784+0.00200588
[5]	train-merror:0.197135+0.00191708	test-merror:0.198313+0.00245247
[6]	train-merror:0.195755+0.00162454	test-merror:0.196647+0.00212877
[7]	train-merror:0.195025+0.00181682	test-merror:0.195567+0.00224116
[8]	train-merror:0.193972+0.00115427	test-merror:0.194513+0.0021932
[9]	train-merror:0.19284+0.0014424	test-merror:0.193506+0.00187807
[10]	train-merror:0.191737+0.00104732	test-merror:0.1923+0.00164097
[11]	train-merror:0.190787+0.00170345	test-merror:0.191307+0.00180553
[12]	train-merror:0.189623+0.00124979	test-merror:0.190454+0.00185665
[13]	train-merror:0.188248+0.00170537	test-merror:0.18916+0.00200566
[14]	train-merror:0.18704+0.00198801	test-m

In [153]:
xgb_nestimators = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, nthread=-1, n_estimators=1040))
train_and_eval(xgb_nestimators, train_x_150_000, le.transform(train_y_150_000), test_x, test_y, log_file_name='150_000.txt')

IndexError: tuple index out of range

200 000 trenovacich dat

In [161]:
train_x_200_000, train_y_200_000 = load_data(FEATURES, TRAIN_SET_NAME, sample=200000)

In [ ]:
xgtrain_200_000 = xgb.DMatrix(train_x_200_000, label=le.transform(train_y_200_000))
xgb_class = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, nthread=10))
cross_validate(xgb_class, xgtrain_200_000, log_file_name='n_estimators_200_000.csv')
#=> n_estimators=1393

In [162]:
xgb_nestimators = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, nthread=8, n_estimators=1393))
train_and_eval(xgb_nestimators, train_x_200_000, le.transform(train_y_200_000), test_x, test_y, log_file_name='200_000.txt')

Done in 48.224149378140766 minutes


všechna data (263 088)

In [193]:
xgtrain = xgb.DMatrix(train_x, label=train_y)
xgb_class = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, nthread=8))
cross_validate(xgb_class, xgtrain, log_file_name='n_estimators_263_088.csv')
#=> n_estimators=1531

In [192]:
xgb_nestimators = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, nthread=8, n_estimators=1531))
train_and_eval(xgb_nestimators, train_x, train_y, test_x, test_y, log_file_name='263_088.txt')

#### Gridsearch

In [9]:
def gridsearch(x, y, log_file_name, tuning_params, model_params, cores=5, xgbthreads=-1):
    start_time = time.time()
    model_params.update(nthread=xgbthreads)
    gsearch = GridSearchCV(
        estimator=XGBClassifier(**dict(XGB_DEFAULT_PARAMS, **model_params)), 
        param_grid=tuning_params, n_jobs=cores, cv=5, verbose=10
    )
    gsearch.fit(x, y)
    pd.DataFrame(gsearch.grid_scores_).to_csv(os.path.join(CV_RESULTS_LOG_PATH, log_file_name))
    print(gsearch.best_estimator_, gsearch.best_score_)
    end_time = time.time()
    print("Done in {} minutes (used cores: {})".format((end_time - start_time)/60, cores))

##### 10 000 dat

maximální hloubka stromu, minimální váha listu

In [ ]:
tuning_params = {
 'max_depth':list(range(3,10,2)),
 'min_child_weight':list(range(1,6,2))
}
model_params = {'n_estimators': 122}
gridsearch(train_x_10000, train_y_10000, 'gs_maxDepth_minChildWeight_010_000.csv', tuning_params, model_params, cores=2)

In [81]:
xgb2_nestimators_dpth_chldW = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, nthread=-1, n_estimators=122, max_depth=9, min_child_weight=1))
train_and_eval(xgb2_nestimators_dpth_chldW, train_x_10000, le.transform(train_y_10000), test_x, test_y, log_file_name='010_000.txt')

gamma

In [101]:
tuning_params_gamma = {'gamma':[i/10.0 for i in range(0,5)]}
model_params = {'n_estimators': 122, 'max_depth':9, 'min_child_weight':1 }
gridsearch(
    train_x_10000, train_y_10000, 'gs_gamma_010_000.csv',
    tuning_params_gamma, model_params, cores=4
)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0.1, learning_rate=0.1, max_delta_step=0, max_depth=9,
       min_child_weight=1, missing=None, n_estimators=122, nthread=4,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8) 0.8471
Done in 4.449427608648936 minutes (used cores: 4)


"vertikalni" a "horizontalni" samplovani

In [102]:
tuning_params_subsample_bytree = {
 'subsample':[i/10.0 for i in range(7,11)],
 'colsample_bytree':[i/10.0 for i in range(7,11)]
}
model_params = {'n_estimators': 122, 'max_depth':9, 'min_child_weight':1, 'gamma':0.1}
gridsearch(
    train_x_10000, train_y_10000, 'gs_subsample_bytree_010_000.csv',
    tuning_params_subsample_bytree, model_params, cores=4
)
#=> 'colsample_bytree':0.8, 'subsample':1,

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0.1, learning_rate=0.1, max_delta_step=0, max_depth=9,
       min_child_weight=1, missing=None, n_estimators=122, nthread=4,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=1.0) 0.8486
Done in 12.811584456761677 minutes (used cores: 4)


novy pocet stromu

In [ ]:
model_params = {'n_estimators': 2000, 'max_depth':9, 'min_child_weight':1, 'gamma':0.1, 'subsample':1, 'colsample_bytree':0.8}
xgb_n_estimators_reduced_rate_010_000 = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, **model_params, nthread=-1))
cross_validate(xgb_n_estimators_reduced_rate_010_000, xgtrain_10000, log_file_name='n_estimators_before_reduced_rate_010_000.csv')

In [124]:
model_params = {'n_estimators': 162, 'max_depth':9, 'min_child_weight':1, 'gamma':0.1, 'subsample':1, 'colsample_bytree':0.8}
xgb2_nestimators_dpth_chldW = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, **model_params, nthread=-1))
train_and_eval(xgb2_nestimators_dpth_chldW, train_x_10000, le.transform(train_y_10000), test_x, test_y, log_file_name='010_000.txt')

In [57]:
model_params = {'n_estimators': 122, 'max_depth':9, 'min_child_weight':1, 'gamma':0.1, 'subsample':1, 'colsample_bytree':0.8}
xgb = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, **model_params, nthread=-1))
train_and_eval(xgb, train_x_10000, le.transform(train_y_10000), test_x, test_y, log_file_name='010_000_orig_estimators.txt')

Done in 0.397456157207489 minutes


změna learning rate - nemá moc cenu, zdá se

In [125]:
model_params = {
    'n_estimators': 2000, 'max_depth':9, 'min_child_weight':1, 'gamma':0.1, 'subsample':1, 'colsample_bytree':0.8,
    'learning_rate': 0.01
}
xgb_n_estimators_reduced_rate_010_000 = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, **model_params, nthread=-1))
cross_validate(xgb_n_estimators_reduced_rate_010_000, xgtrain_10000, log_file_name='n_estimators_reduced_rate_010_000.csv')

[0]	train-merror:0.1269+0.00466329	test-merror:0.202+0.0129035
[1]	train-merror:0.108925+0.00226053	test-merror:0.1923+0.013611
[2]	train-merror:0.102125+0.00178536	test-merror:0.187+0.0116705
[3]	train-merror:0.098775+0.00429782	test-merror:0.183+0.0117771
[4]	train-merror:0.0999+0.00456166	test-merror:0.1824+0.0100717
[5]	train-merror:0.096725+0.00535922	test-merror:0.18+0.0120125
[6]	train-merror:0.0965+0.0054778	test-merror:0.1801+0.00963016
[7]	train-merror:0.094625+0.00378071	test-merror:0.1774+0.0101361
[8]	train-merror:0.094475+0.0048903	test-merror:0.1778+0.0108885
[9]	train-merror:0.094175+0.00456385	test-merror:0.1768+0.0108839
[10]	train-merror:0.093925+0.00319824	test-merror:0.1758+0.0118178
[11]	train-merror:0.094625+0.00311047	test-merror:0.1762+0.0112009
[12]	train-merror:0.0943+0.00360694	test-merror:0.1782+0.00947945
[13]	train-merror:0.094325+0.00282356	test-merror:0.1768+0.00896437
[14]	train-merror:0.093825+0.00305941	test-merror:0.176+0.00932738
[15]	train-merror:

In [36]:
model_params = {
    'n_estimators':257 , 'max_depth':9, 'min_child_weight':1, 'gamma':0.1, 'subsample':1, 'colsample_bytree':0.8,
    'learning_rate': 0.01
}
xgb2_nestimators_dpth_chldW = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, **model_params, nthread=-1))
train_and_eval(xgb2_nestimators_dpth_chldW, train_x_10000, le.transform(train_y_10000), test_x, test_y, log_file_name='010_000.txt')

SyntaxError: invalid syntax (<ipython-input-36-bd202c20cf9d>, line 5)

##### 20 000 dat

In [109]:
tuning_params = {
    'max_depth':list(range(3,10,2)),
    'min_child_weight':list(range(1,6,2))
}
model_params = {'n_estimators': 271}
gridsearch(train_x_20000, train_y_20000, 'gs_maxDepth_minChildWeight_020_000.csv', tuning_params, model_params, cores=5)
#=> 'max_depth':7, 'min_child_weight':1

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] max_depth=3, min_child_weight=1 .................................
[CV] max_depth=3, min_child_weight=1 .................................
[CV] max_depth=3, min_child_weight=1 .................................
[CV] max_depth=3, min_child_weight=1 .................................
[CV] max_depth=3, min_child_weight=1 .................................
[CV] ........ max_depth=3, min_child_weight=1, score=0.859035 -  54.4s
[CV] max_depth=3, min_child_weight=3 .................................
[CV] ........ max_depth=3, min_child_weight=1, score=0.861431 -  58.8s
[CV] max_depth=3, min_child_weight=3 .................................
[CV] ........ max_depth=3, min_child_weight=1, score=0.844039 -  59.7s
[CV] max_depth=3, min_child_weight=3 .................................


[Parallel(n_jobs=5)]: Done   3 tasks      | elapsed:   59.9s


[CV] ........ max_depth=3, min_child_weight=1, score=0.854714 - 1.0min
[CV] max_depth=3, min_child_weight=3 .................................
[CV] ........ max_depth=3, min_child_weight=1, score=0.857286 - 1.0min
[CV] max_depth=3, min_child_weight=3 .................................
[CV] ........ max_depth=3, min_child_weight=3, score=0.845289 -  56.9s
[CV] max_depth=3, min_child_weight=5 .................................
[CV] ........ max_depth=3, min_child_weight=3, score=0.855036 -  57.5s
[CV] max_depth=3, min_child_weight=5 .................................
[CV] ........ max_depth=3, min_child_weight=3, score=0.855786 - 1.0min
[CV] max_depth=3, min_child_weight=5 .................................


[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:  2.0min


[CV] ........ max_depth=3, min_child_weight=3, score=0.863182 -  59.0s
[CV] max_depth=3, min_child_weight=5 .................................
[CV] ........ max_depth=3, min_child_weight=3, score=0.858465 - 1.0min
[CV] max_depth=3, min_child_weight=5 .................................
[CV] ........ max_depth=3, min_child_weight=5, score=0.843289 -  57.0s
[CV] max_depth=5, min_child_weight=1 .................................
[CV] ........ max_depth=3, min_child_weight=5, score=0.857786 -  56.8s
[CV] max_depth=5, min_child_weight=1 .................................
[CV] ........ max_depth=3, min_child_weight=5, score=0.857536 -  57.5s
[CV] max_depth=5, min_child_weight=1 .................................
[CV] ........ max_depth=3, min_child_weight=5, score=0.854714 -  58.1s
[CV] max_depth=5, min_child_weight=1 .................................
[CV] ........ max_depth=3, min_child_weight=5, score=0.856178 -  59.1s
[CV] max_depth=5, min_child_weight=1 .................................


...

[CV] ........ max_depth=9, min_child_weight=3, score=0.861785 - 2.5min
[CV] max_depth=9, min_child_weight=5 .................................
[CV] ........ max_depth=9, min_child_weight=3, score=0.868783 - 2.6min
[CV] max_depth=9, min_child_weight=5 .................................
[CV] ........ max_depth=9, min_child_weight=3, score=0.871218 - 2.6min
[CV] max_depth=9, min_child_weight=5 .................................
[CV] ........ max_depth=9, min_child_weight=3, score=0.865933 - 2.5min
[CV] max_depth=9, min_child_weight=5 .................................
[CV] ........ max_depth=9, min_child_weight=5, score=0.855786 - 2.6min
[CV] ........ max_depth=9, min_child_weight=5, score=0.869033 - 2.6min
[CV] ........ max_depth=9, min_child_weight=5, score=0.859535 - 2.6min


[Parallel(n_jobs=5)]: Done  58 out of  60 | elapsed: 21.1min remaining:   43.7s


[CV] ........ max_depth=9, min_child_weight=5, score=0.866183 - 2.5min
[CV] ........ max_depth=9, min_child_weight=5, score=0.866467 - 2.6min


[Parallel(n_jobs=5)]: Done  60 out of  60 | elapsed: 21.2min finished


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=7,
       min_child_weight=1, missing=None, n_estimators=271, nthread=5,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8) 0.8684
Done in 22.479429133733113 minutes (used cores: 5)


In [167]:
tuning_params_gamma = {'gamma':[i/10.0 for i in range(0,6)]}
model_params = {'n_estimators': 271, 'max_depth':7, 'min_child_weight':1 }
gridsearch(
    train_x_20000, train_y_20000, 'gs_gamma_020_000.csv',
    tuning_params_gamma, model_params, cores=1, xgbthreads=10
)
#=> 'gamma':0.4

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] gamma=0.4 .......................................................
[CV] .............................. gamma=0.4, score=0.860785 - 1.5min
[CV] gamma=0.4 .......................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.5min remaining:    0.0s


[CV] .............................. gamma=0.4, score=0.875031 - 1.5min
[CV] gamma=0.4 .......................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.9min remaining:    0.0s


[CV] .............................. gamma=0.4, score=0.868533 - 1.5min
[CV] gamma=0.4 .......................................................


...

[CV] .............................. gamma=0.5, score=0.870468 - 1.5min
[CV] gamma=0.5 .......................................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 13.2min remaining:    0.0s


[CV] .............................. gamma=0.5, score=0.875438 - 1.5min


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed: 14.7min finished


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0.4, learning_rate=0.1, max_delta_step=0, max_depth=7,
       min_child_weight=1, missing=None, n_estimators=271, nthread=10,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8) 0.8706
Done in 16.402488521734874 minutes (used cores: 1)


In [168]:
tuning_params_subsample_bytree = {
 'subsample':[i/10.0 for i in range(7,11)],
 'colsample_bytree':[i/10.0 for i in range(7,11)]
}
model_params = {'n_estimators': 271, 'max_depth':7, 'min_child_weight':1, 'gamma':0.4}
gridsearch(
    train_x_20000, train_y_20000, 'gs_subsample_bytree_020_000.csv',
    tuning_params_subsample_bytree, model_params, cores=1, xgbthreads=10
)
#=> 'colsample_bytree':1, 'subsample':0.9,

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] colsample_bytree=0.7, subsample=0.7 .............................
[CV] .... colsample_bytree=0.7, subsample=0.7, score=0.861285 - 1.4min
[CV] colsample_bytree=0.7, subsample=0.7 .............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.4min remaining:    0.0s


[CV] .... colsample_bytree=0.7, subsample=0.7, score=0.869033 - 1.4min
[CV] colsample_bytree=0.7, subsample=0.7 .............................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.8min remaining:    0.0s


[CV] .... colsample_bytree=0.7, subsample=0.7, score=0.870782 - 1.4min
[CV] colsample_bytree=0.7, subsample=0.7 .............................


...

[CV] .... colsample_bytree=0.7, subsample=0.8, score=0.870468 -  48.4s
[CV] colsample_bytree=0.7, subsample=0.8 .............................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  9.3min remaining:    0.0s


[CV] .... colsample_bytree=0.7, subsample=0.8, score=0.872186 -  48.7s
[CV] colsample_bytree=0.7, subsample=0.9 .............................
[CV] .... colsample_bytree=0.7, subsample=0.9, score=0.865284 -  47.2s
[CV] colsample_bytree=0.7, subsample=0.9 .............................
[CV] .... colsample_bytree=0.7, subsample=0.9, score=0.873782 -  47.0s
[CV] colsample_bytree=0.7, subsample=0.9 .............................
[CV] .... colsample_bytree=0.7, subsample=0.9, score=0.869783 -  47.1s
[CV] colsample_bytree=0.7, subsample=0.9 .............................
[CV] .... colsample_bytree=0.7, subsample=0.9, score=0.872218 -  46.9s
[CV] colsample_bytree=0.7, subsample=0.9 .............................
[CV] .... colsample_bytree=0.7, subsample=0.9, score=0.873187 -  47.0s
[CV] colsample_bytree=0.7, subsample=1.0 .............................
[CV] .... colsample_bytree=0.7, subsample=1.0, score=0.866533 -  45.0s
[CV] colsample_bytree=0.7, subsample=1.0 .............................
[CV] .

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed: 102.3min finished


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.7,
       gamma=0.4, learning_rate=0.1, max_delta_step=0, max_depth=7,
       min_child_weight=1, missing=None, n_estimators=271, nthread=10,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.9) 0.87085
Done in 103.83961979945501 minutes (used cores: 1)


In [170]:
model_params = {
    'n_estimators': 2000, 'max_depth':7, 'min_child_weight':1, 'gamma': 0.4, 'subsample':0.9, 'colsample_bytree':1,
    'nthread': -1
}
xgb_n_estimators_reduced_rate_020_000 = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, **model_params))
cross_validate(xgb_n_estimators_reduced_rate_020_000, xgtrain_20000, log_file_name='n_estimators_before_reduced_rate_020_000.csv')
#=> 'n_estimators': 198

[0]	train-merror:0.175613+0.00290467	test-merror:0.1955+0.00304959
[1]	train-merror:0.167975+0.00315635	test-merror:0.1887+0.00288704
[2]	train-merror:0.162412+0.00308598	test-merror:0.1847+0.00275409
[3]	train-merror:0.160175+0.00344914	test-merror:0.18235+0.00427668
[4]	train-merror:0.157625+0.00227042	test-merror:0.17995+0.00388716
[5]	train-merror:0.156162+0.00244481	test-merror:0.18005+0.00293854
[6]	train-merror:0.15465+0.00241934	test-merror:0.1789+0.0025865
[7]	train-merror:0.151975+0.00205327	test-merror:0.17725+0.00270647
[8]	train-merror:0.150063+0.00199775	test-merror:0.1761+0.00299833
[9]	train-merror:0.148312+0.00179336	test-merror:0.1748+0.00267582
[10]	train-merror:0.146112+0.00197173	test-merror:0.1739+0.00217141
[11]	train-merror:0.144175+0.00195512	test-merror:0.17275+0.00153297
[12]	train-merror:0.1421+0.00165213	test-merror:0.173+0.0018303
[13]	train-merror:0.140263+0.00178199	test-merror:0.1713+0.00195832
[14]	train-merror:0.138413+0.00209477	test-merror:0.17065+0

In [172]:
model_params = {
    'n_estimators': 198, 'max_depth':7, 'min_child_weight':1, 'gamma': 0.4, 'subsample':0.9, 'colsample_bytree':1,
    'nthread': -1
}
xgb_tuned = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, **model_params))
train_and_eval(xgb_tuned, train_x_20000, le.transform(train_y_20000), test_x, test_y, log_file_name='020_000.txt')

Done in 1.8083741188049316 minutes


In [55]:
model_params = {
    'n_estimators': 271, 'max_depth':7, 'min_child_weight':1, 'gamma': 0.4, 'subsample':0.9, 'colsample_bytree':1,
    'nthread': -1
}
xgb_tuned = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, **model_params))
train_and_eval(xgb_tuned, train_x_20000, le.transform(train_y_20000), test_x, test_y, log_file_name='020_000_orig_estimators.txt')

Done in 1.293818680445353 minutes


##### 50 000 dat

In [173]:
tuning_params = {
    'max_depth':list(range(9,12,2)),
    #'max_depth':list(range(3,10,2)),
    #'min_child_weight':list(range(1,6,2))
}
model_params = {'n_estimators': 596}
gridsearch(
    train_x_50000, train_y_50000, 'gs_maxDepth_minChildWeight_050_000_.csv', tuning_params, model_params, cores=1,
    xgbthreads=10
)
#=> 'max_depth':9, 'min_child_weight':1,

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] max_depth=9 .....................................................
[CV] ............................ max_depth=9, score=0.888100 - 8.1min
[CV] max_depth=9 .....................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  8.1min remaining:    0.0s


[CV] ............................ max_depth=9, score=0.888400 - 8.1min
[CV] max_depth=9 .....................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 16.2min remaining:    0.0s


[CV] ............................ max_depth=9, score=0.890600 - 8.1min
[CV] max_depth=9 .....................................................


...

[CV] ........................... max_depth=11, score=0.884800 - 5.9min
[CV] max_depth=11 ....................................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 63.9min remaining:    0.0s


[CV] ........................... max_depth=11, score=0.881200 - 5.9min


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed: 69.8min finished


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=9,
       min_child_weight=1, missing=None, n_estimators=596, nthread=10,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8) 0.88784
Done in 75.66002370516459 minutes (used cores: 1)


gamma

In [141]:
tuning_params_gamma = {'gamma':[i/10.0 for i in range(0,5)]}
model_params = {'n_estimators': 596, 'max_depth':9, 'min_child_weight':1 }
gridsearch(
    train_x_50000, train_y_50000, 'gs_gamma_050_000.csv',
    tuning_params_gamma, model_params, cores=5
)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] gamma=0.0 .......................................................
[CV] gamma=0.0 .......................................................
[CV] gamma=0.0 .......................................................
[CV] gamma=0.0 .......................................................
[CV] gamma=0.0 .......................................................
[CV] .............................. gamma=0.0, score=0.890600 -14.4min
[CV] gamma=0.1 .......................................................
[CV] .............................. gamma=0.0, score=0.885500 -14.5min
[CV] gamma=0.1 .......................................................
[CV] .............................. gamma=0.0, score=0.886600 -14.6min
[CV] gamma=0.1 .......................................................


[Parallel(n_jobs=5)]: Done   3 tasks      | elapsed: 14.7min


[CV] .............................. gamma=0.0, score=0.888400 -14.8min
[CV] gamma=0.1 .......................................................
[CV] .............................. gamma=0.0, score=0.888100 -15.0min
[CV] gamma=0.1 .......................................................
[CV] .............................. gamma=0.1, score=0.885900 -23.1min
[CV] gamma=0.2 .......................................................
[CV] .............................. gamma=0.1, score=0.887100 -23.0min
[CV] gamma=0.2 .......................................................
[CV] .............................. gamma=0.1, score=0.889300 -23.7min
[CV] gamma=0.2 .......................................................


[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed: 38.3min


[CV] .............................. gamma=0.1, score=0.887500 -23.6min
[CV] gamma=0.2 .......................................................
[CV] .............................. gamma=0.1, score=0.883800 -23.5min
[CV] gamma=0.2 .......................................................
[CV] .............................. gamma=0.2, score=0.889800 -23.7min
[CV] gamma=0.3 .......................................................
[CV] .............................. gamma=0.2, score=0.887200 -23.0min
[CV] gamma=0.3 .......................................................
[CV] .............................. gamma=0.2, score=0.887700 -24.1min
[CV] gamma=0.3 .......................................................
[CV] .............................. gamma=0.2, score=0.885300 -23.3min
[CV] gamma=0.3 .......................................................
[CV] .............................. gamma=0.2, score=0.887800 -23.5min
[CV] gamma=0.3 .......................................................


...

[CV] .............................. gamma=0.3, score=0.885700 -19.2min
[CV] gamma=0.4 .......................................................
[CV] .............................. gamma=0.4, score=0.886300 -15.0min
[CV] .............................. gamma=0.4, score=0.889900 -15.1min


[Parallel(n_jobs=5)]: Done  22 out of  25 | elapsed: 95.0min remaining: 13.0min


[CV] .............................. gamma=0.4, score=0.890800 -15.1min
[CV] .............................. gamma=0.4, score=0.885100 -15.1min
[CV] .............................. gamma=0.4, score=0.885900 -14.7min


[Parallel(n_jobs=5)]: Done  25 out of  25 | elapsed: 95.8min remaining:    0.0s
[Parallel(n_jobs=5)]: Done  25 out of  25 | elapsed: 95.8min finished


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0.0, learning_rate=0.1, max_delta_step=0, max_depth=9,
       min_child_weight=1, missing=None, n_estimators=596, nthread=5,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8) 0.88784
Done in 105.92381353378296 minutes (used cores: 5)


In [ ]:
tuning_params_subsample_bytree = {
 'subsample':[i/10.0 for i in range(8,11)],
 'colsample_bytree':[i/10.0 for i in range(8,11)]
}
model_params = {'n_estimators': 596, 'max_depth':9, 'min_child_weight':1, 'gamma':0}
gridsearch(
    train_x_50000, train_y_50000, 'gs_subsample_bytree_050_000.csv',
    tuning_params_subsample_bytree, model_params, cores=2
)
#=> 'colsample_bytree':1, 'subsample':0.9,

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] colsample_bytree=0.8, subsample=0.8 .............................
[CV] colsample_bytree=0.8, subsample=0.8 .............................
[CV] .... colsample_bytree=0.8, subsample=0.8, score=0.888100 -23.0min
[CV] colsample_bytree=0.8, subsample=0.8 .............................


[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed: 23.1min


[CV] .... colsample_bytree=0.8, subsample=0.8, score=0.888400 -23.2min
[CV] colsample_bytree=0.8, subsample=0.8 .............................
[CV] .... colsample_bytree=0.8, subsample=0.8, score=0.886600 -17.6min
[CV] colsample_bytree=0.8, subsample=0.8 .............................
[CV] .... colsample_bytree=0.8, subsample=0.8, score=0.890600 -19.2min
[CV] colsample_bytree=0.8, subsample=0.9 .............................


[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed: 42.2min


[CV] .... colsample_bytree=0.8, subsample=0.8, score=0.885500 -19.1min
[CV] colsample_bytree=0.8, subsample=0.9 .............................
[CV] .... colsample_bytree=0.8, subsample=0.9, score=0.885900 -17.7min
[CV] colsample_bytree=0.8, subsample=0.9 .............................
[CV] .... colsample_bytree=0.8, subsample=0.9, score=0.890400 -23.7min
[CV] colsample_bytree=0.8, subsample=0.9 .............................
[CV] .... colsample_bytree=0.8, subsample=0.9, score=0.889700 -24.0min
[CV] colsample_bytree=0.8, subsample=0.9 .............................
[CV] .... colsample_bytree=0.8, subsample=0.9, score=0.887400 -21.8min
[CV] colsample_bytree=0.8, subsample=1.0 .............................


...

[CV] .... colsample_bytree=0.9, subsample=0.9, score=0.890000 -24.2min
[CV] colsample_bytree=0.9, subsample=0.9 .............................
[CV] .... colsample_bytree=0.9, subsample=0.9, score=0.890400 -26.3min
[CV] colsample_bytree=0.9, subsample=0.9 .............................
[CV] .... colsample_bytree=0.9, subsample=0.9, score=0.887200 -26.8min
[CV] colsample_bytree=0.9, subsample=1.0 .............................
[CV] .... colsample_bytree=0.9, subsample=0.9, score=0.886900 -24.6min
[CV] colsample_bytree=0.9, subsample=1.0 .............................
[CV] .... colsample_bytree=0.9, subsample=1.0, score=0.885600 -19.2min
[CV] colsample_bytree=0.9, subsample=1.0 .............................
[CV] .... colsample_bytree=0.9, subsample=1.0, score=0.888800 -17.5min
[CV] colsample_bytree=0.9, subsample=1.0 .............................
[CV] .... colsample_bytree=0.9, subsample=1.0, score=0.891200 -18.4min
[CV] colsample_bytree=0.9, subsample=1.0 .............................


[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed: 302.8min


[CV] .... colsample_bytree=0.9, subsample=1.0, score=0.885100 -20.7min
[CV] colsample_bytree=1.0, subsample=0.8 .............................
[CV] .... colsample_bytree=0.9, subsample=1.0, score=0.888100 -24.2min
[CV] colsample_bytree=1.0, subsample=0.8 .............................
[CV] .... colsample_bytree=1.0, subsample=0.8, score=0.884000 -30.8min
[CV] colsample_bytree=1.0, subsample=0.8 .............................
[CV] .... colsample_bytree=1.0, subsample=0.8, score=0.888500 -27.7min
[CV] colsample_bytree=1.0, subsample=0.8 .............................
[CV] .... colsample_bytree=1.0, subsample=0.8, score=0.888600 -22.7min
[CV] colsample_bytree=1.0, subsample=0.8 .............................
[CV] .... colsample_bytree=1.0, subsample=0.8, score=0.887100 -23.6min
[CV] colsample_bytree=1.0, subsample=0.9 .............................
[CV] .... colsample_bytree=1.0, subsample=0.8, score=0.885600 -22.5min
[CV] colsample_bytree=1.0, subsample=0.9 .............................
[CV] .

[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed: 418.1min


[CV] .... colsample_bytree=1.0, subsample=0.9, score=0.890900 -28.4min
[CV] colsample_bytree=1.0, subsample=0.9 .............................
[CV] .... colsample_bytree=1.0, subsample=0.9, score=0.889200 -25.2min
[CV] colsample_bytree=1.0, subsample=1.0 .............................
[CV] .... colsample_bytree=1.0, subsample=0.9, score=0.886100 -20.8min
[CV] colsample_bytree=1.0, subsample=1.0 .............................


In [54]:
model_params = {
    'n_estimators': 2000, 'max_depth':9, 'min_child_weight':1, 'gamma':0, 'subsample':0.9, 'colsample_bytree':1,
    'nthread': -1
}
xgb_n_estimators_reduced_rate_050_000 = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, **model_params))
cross_validate(xgb_n_estimators_reduced_rate_050_000, xgtrain_50000, log_file_name='n_estimators_before_reduced_rate_050_000.csv')
#=> 'n_estimators': 354

[0]	train-merror:0.158505+0.00367918	test-merror:0.17714+0.00509533
[1]	train-merror:0.149205+0.00266155	test-merror:0.16828+0.00698123
[2]	train-merror:0.14622+0.00175879	test-merror:0.16602+0.00540348
[3]	train-merror:0.14304+0.00144876	test-merror:0.1633+0.00508842
[4]	train-merror:0.14051+0.00119484	test-merror:0.16192+0.0054389
[5]	train-merror:0.137595+0.00132848	test-merror:0.15924+0.0056666
[6]	train-merror:0.13551+0.00124866	test-merror:0.15868+0.00503325
[7]	train-merror:0.13216+0.00167747	test-merror:0.15682+0.00560264
[8]	train-merror:0.12975+0.0016961	test-merror:0.15488+0.00505743
[9]	train-merror:0.12743+0.00163534	test-merror:0.15394+0.00548109
[10]	train-merror:0.12483+0.0017617	test-merror:0.15296+0.00522096
[11]	train-merror:0.122575+0.00187697	test-merror:0.15196+0.0059972
[12]	train-merror:0.12091+0.00225819	test-merror:0.15158+0.00546824
[13]	train-merror:0.119545+0.00148243	test-merror:0.1506+0.00572538
[14]	train-merror:0.11812+0.00143094	test-merror:0.14964+0.0

In [55]:
model_params = {
    'n_estimators': 354, 'max_depth':9, 'min_child_weight':1, 'gamma':0, 'subsample':0.9, 'colsample_bytree':1,
    'nthread': -1
}
xgb_tuned = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, **model_params))
train_and_eval(xgb_tuned, train_x_50000, le.transform(train_y_50000), test_x, test_y, log_file_name='050_000.txt')

Done in 6.647509817282359 minutes


In [52]:
model_params = {
    'n_estimators': 596, 'max_depth':9, 'min_child_weight':1, 'gamma':0, 'subsample':0.9, 'colsample_bytree':1,
    'nthread': 8
}
xgb_tuned = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, **model_params))
train_and_eval(xgb_tuned, train_x_50000, le.transform(train_y_50000), test_x, test_y, log_file_name='050_000_orig_estimators.txt')

Done in 7.96691507101059 minutes


##### 100 000 dat

In [ ]:
tuning_params = {
    'max_depth':list(range(5,10,2)),
    'min_child_weight':list(range(1,6,2))
}
model_params = {'n_estimators': 790}
gridsearch(train_x_100_000, train_y_100_000, 'gs_maxDepth_minChildWeight_100_000.csv', tuning_params, model_params, cores=5)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] max_depth=5, min_child_weight=1 .................................
[CV] max_depth=5, min_child_weight=1 .................................
[CV] max_depth=5, min_child_weight=1 .................................
[CV] max_depth=5, min_child_weight=1 .................................
[CV] max_depth=5, min_child_weight=1 .................................
[CV] max_depth=5, min_child_weight=3 .................................
[CV] max_depth=5, min_child_weight=3 .................................
[CV] max_depth=5, min_child_weight=3 .................................
[CV] max_depth=5, min_child_weight=3 .................................
[CV] max_depth=5, min_child_weight=3 .................................


In [ ]:
tuning_params_gamma = {'gamma':[i/10.0 for i in range(0,5)]}
model_params = {'n_estimators': 790, 'max_depth': 9, 'min_child_weight': 1}
gridsearch(
    train_x_100_000, train_y_100_000, 'gs_gamma_100_000.csv',
    tuning_params_gamma, model_params, cores=1, xgbthreads=-1
)

##### 263 088 dat

In [181]:
tuning_params = {
    'max_depth': list(range(7,10,2)),
    'min_child_weight': list(range(1,4,2))
}
model_params = {'n_estimators': 1531}
gridsearch(train_x, train_y, 'gs_maxDepth_minChildWeight_263_088_.csv', tuning_params, model_params,
           cores=1, xgbthreads=-1)
#=> 'max_depth': 11, 'min_child_weight':1, (762 min)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] max_depth=13, min_child_weight=1 ................................
[CV] ....... max_depth=13, min_child_weight=1, score=0.907410 -54.7min
[CV] max_depth=13, min_child_weight=1 ................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 54.7min remaining:    0.0s


[CV] ....... max_depth=13, min_child_weight=1, score=0.912520 -54.7min
[CV] max_depth=13, min_child_weight=1 ................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 109.4min remaining:    0.0s


[CV] ....... max_depth=13, min_child_weight=1, score=0.913832 -54.6min
[CV] max_depth=13, min_child_weight=1 ................................


...

[CV] ....... max_depth=13, min_child_weight=1, score=0.909060 -54.7min
[CV] max_depth=13, min_child_weight=1 ................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 218.7min remaining:    0.0s


[CV] ....... max_depth=13, min_child_weight=1, score=0.910198 -54.6min


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 273.3min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 273.3min finished


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=13,
       min_child_weight=1, missing=None, n_estimators=1531, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8) 0.9106040564373897
Done in 340.6876859664917 minutes (used cores: 1)


In [186]:
tuning_params_gamma = {'gamma': [0.3]} #[i/10.0 for i in range(0,6)]} # 0 je defaultni hodnota, skore tedy uz mame
model_params = {'n_estimators': 1531, 'max_depth': 11, 'min_child_weight':1}
gridsearch(
    train_x, train_y, 'gs_gamma_263_088.csv',
    tuning_params_gamma, model_params, cores=1, xgbthreads=-1
)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] gamma=0.3 .......................................................
[CV] .............................. gamma=0.3, score=0.908056 -46.4min
[CV] gamma=0.3 .......................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 46.4min remaining:    0.0s


[CV] .............................. gamma=0.3, score=0.912672 -46.1min
[CV] gamma=0.3 .......................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 92.5min remaining:    0.0s


[CV] .............................. gamma=0.3, score=0.914288 -46.8min
[CV] gamma=0.3 .......................................................


...

[CV] .............................. gamma=0.3, score=0.909573 -46.2min
[CV] gamma=0.3 .......................................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 185.6min remaining:    0.0s


[CV] .............................. gamma=0.3, score=0.910883 -46.1min


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 231.7min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 231.7min finished


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0.3, learning_rate=0.1, max_delta_step=0, max_depth=11,
       min_child_weight=1, missing=None, n_estimators=1531, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8) 0.9110943866690994
Done in 288.8138397494952 minutes (used cores: 1)


In [187]:
tuning_params_gamma = {'gamma': [0.4]}
model_params = {'n_estimators': 1531, 'max_depth': 11, 'min_child_weight':1}
gridsearch(
    train_x, train_y, 'gs_gamma_263_088_04.csv',
    tuning_params_gamma, model_params, cores=1, xgbthreads=-1
)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] gamma=0.4 .......................................................
[CV] .............................. gamma=0.4, score=0.908341 -46.1min
[CV] gamma=0.4 .......................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 46.2min remaining:    0.0s


[CV] .............................. gamma=0.4, score=0.912520 -46.2min
[CV] gamma=0.4 .......................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 92.4min remaining:    0.0s


[CV] .............................. gamma=0.4, score=0.914079 -46.2min
[CV] gamma=0.4 .......................................................


...

[CV] .............................. gamma=0.4, score=0.909725 -46.3min
[CV] gamma=0.4 .......................................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 184.9min remaining:    0.0s


[CV] .............................. gamma=0.4, score=0.910293 -46.0min


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 231.0min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 231.0min finished


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0.4, learning_rate=0.1, max_delta_step=0, max_depth=11,
       min_child_weight=1, missing=None, n_estimators=1531, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8) 0.9109917594112996
Done in 288.2789105971654 minutes (used cores: 1)


In [188]:
tuning_params_gamma = {'gamma': [0.5]}
model_params = {'n_estimators': 1531, 'max_depth': 11, 'min_child_weight':1}
gridsearch(
    train_x, train_y, 'gs_gamma_263_088_05.csv',
    tuning_params_gamma, model_params, cores=1, xgbthreads=-1
)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] gamma=0.5 .......................................................
[CV] .............................. gamma=0.5, score=0.908227 -46.2min
[CV] gamma=0.5 .......................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 46.2min remaining:    0.0s


[CV] .............................. gamma=0.5, score=0.912958 -46.0min
[CV] gamma=0.5 .......................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 92.3min remaining:    0.0s


[CV] .............................. gamma=0.5, score=0.914383 -46.3min
[CV] gamma=0.5 .......................................................


...

[CV] .............................. gamma=0.5, score=0.909896 -46.2min
[CV] gamma=0.5 .......................................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 184.8min remaining:    0.0s


[CV] .............................. gamma=0.5, score=0.910997 -46.1min


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 230.9min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 230.9min finished


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0.5, learning_rate=0.1, max_delta_step=0, max_depth=11,
       min_child_weight=1, missing=None, n_estimators=1531, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8) 0.9112920391656024
Done in 287.907623565197 minutes (used cores: 1)


In [189]:
tuning_params_gamma = {'gamma': [0.6]}
model_params = {'n_estimators': 1531, 'max_depth': 11, 'min_child_weight':1}
gridsearch(
    train_x, train_y, 'gs_gamma_263_088_06.csv',
    tuning_params_gamma, model_params, cores=1, xgbthreads=-1
)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] gamma=0.6 .......................................................
[CV] .............................. gamma=0.6, score=0.908569 -46.0min
[CV] gamma=0.6 .......................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 46.0min remaining:    0.0s


[CV] .............................. gamma=0.6, score=0.913680 -46.1min
[CV] gamma=0.6 .......................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 92.1min remaining:    0.0s


[CV] .............................. gamma=0.6, score=0.914668 -46.3min
[CV] gamma=0.6 .......................................................


...

[CV] .............................. gamma=0.6, score=0.909706 -46.3min
[CV] gamma=0.6 .......................................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 184.8min remaining:    0.0s


[CV] .............................. gamma=0.6, score=0.911377 -46.0min


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 230.8min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 230.8min finished


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0.6, learning_rate=0.1, max_delta_step=0, max_depth=11,
       min_child_weight=1, missing=None, n_estimators=1531, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8) 0.9115999209390014
Done in 287.6256788055102 minutes (used cores: 1)


In [190]:
tuning_params_gamma = {'gamma': [0.7]}
model_params = {'n_estimators': 1531, 'max_depth': 11, 'min_child_weight':1}
gridsearch(
    train_x, train_y, 'gs_gamma_263_088_07.csv',
    tuning_params_gamma, model_params, cores=1, xgbthreads=-1
)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] gamma=0.7 .......................................................
[CV] .............................. gamma=0.7, score=0.908645 -46.0min
[CV] gamma=0.7 .......................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 46.0min remaining:    0.0s


[CV] .............................. gamma=0.7, score=0.912939 -46.2min
[CV] gamma=0.7 .......................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 92.3min remaining:    0.0s


[CV] .............................. gamma=0.7, score=0.913965 -46.4min
[CV] gamma=0.7 .......................................................


...

[CV] .............................. gamma=0.7, score=0.910618 -46.3min
[CV] gamma=0.7 .......................................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 185.0min remaining:    0.0s


[CV] .............................. gamma=0.7, score=0.911396 -46.0min


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 230.9min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 230.9min finished


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0.7, learning_rate=0.1, max_delta_step=0, max_depth=11,
       min_child_weight=1, missing=None, n_estimators=1531, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8) 0.9115124977193942
Done in 288.2401100039482 minutes (used cores: 1)


In [16]:
model_params = {'n_estimators': 1531, 'max_depth': 11, 'min_child_weight':1, 'gamma':0.6}
tuning_params = {
 'subsample':[1],#[i/10.0 for i in range(8,11)],
 'colsample_bytree':[0.9]#[i/10.0 for i in range(9,11)]
}
gridsearch(
    train_x, train_y, 'gs_subsample_bytree_263_088_9_10.csv',
    tuning_params, model_params, cores=1, xgbthreads=8
)
#=> 'colsample_bytree': 0.9, 'subsample': 0.9

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] colsample_bytree=0.9, subsample=1 ...............................


KeyboardInterrupt: 

In [14]:
model_params = {
    'n_estimators': 2000, 'max_depth': 11, 'min_child_weight':1, 'gamma':0.6, 'subsample':0.9, 'colsample_bytree':0.9,
    'nthread': -1
}
xgb_model = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, **model_params))
cross_validate(xgb_model, xgtrain, log_file_name='n_estimators_before_reduced_rate_263_088.csv')
#=> 'n_estimators': 705 (229 minutes)

[0]	train-merror:0.146487+0.00157591	test-merror:0.154756+0.00122915
[1]	train-merror:0.136796+0.00110853	test-merror:0.145896+0.000906983
[2]	train-merror:0.133522+0.0011245	test-merror:0.143022+0.000647996
[3]	train-merror:0.131103+0.00127833	test-merror:0.140977+0.000685097
[4]	train-merror:0.129556+0.00145586	test-merror:0.139316+0.000874686
[5]	train-merror:0.127345+0.000746158	test-merror:0.137302+0.000730707
[6]	train-merror:0.124821+0.000720059	test-merror:0.13539+0.000966151
[7]	train-merror:0.122486+0.000433688	test-merror:0.133801+0.000811583
[8]	train-merror:0.120626+0.000488203	test-merror:0.132052+0.000712335
[9]	train-merror:0.119258+0.000628997	test-merror:0.131269+0.000487114
[10]	train-merror:0.117724+0.000372001	test-merror:0.130091+0.000593701
[11]	train-merror:0.115992+0.000393475	test-merror:0.128913+0.00070872
[12]	train-merror:0.114356+0.000766777	test-merror:0.127563+0.000854571
[13]	train-merror:0.11326+0.000742434	test-merror:0.12686+0.00103193
[14]	train-mer

In [18]:
model_params = {
    'n_estimators': 705, 'max_depth': 11, 'min_child_weight':1, 'gamma':0.6, 'subsample':0.9, 'colsample_bytree':0.9,
    'nthread': 8
}
xgb_tuned = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, **model_params))
train_and_eval(xgb_tuned, train_x, le.transform(train_y), test_x, test_y, log_file_name='263_088.txt')

Done in 52.344903298219045 minutes


In [19]:
model_params = {
    'n_estimators': 1531, 'max_depth': 11, 'min_child_weight':1, 'gamma':0.6, 'subsample':0.9, 'colsample_bytree':0.9,
    'nthread': 8
}
xgb_tuned = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, **model_params))
train_and_eval(xgb_tuned, train_x, le.transform(train_y), test_x, test_y, log_file_name='263_088_orig_estimators.txt')

Done in 116.61845057408014 minutes


znovu prepocitame pocet stromu, gamma:0.6 je uz dost odlisny od defaultni hodnoty, tak se radsi mrknem, jestli nam to cely neposune nekam jinam. Mozna je to ale blbost, anzto mame paramtetry uz potuneny na neco jinyho...

In [195]:
model_params = {
    'n_estimators': 2000, 'max_depth':11, 'min_child_weight':1, 'gamma':0.6,
    'nthread': -1
}
xgb_n_estimators = XGBClassifier(**dict(XGB_DEFAULT_PARAMS, **model_params))
cross_validate(xgb_n_estimators, xgtrain, log_file_name='n_estimators_after_gamma_263_088.csv')
#=> 'n_estimators': 631

[0]	train-merror:0.148912+0.00198479	test-merror:0.15595+0.00157893
[1]	train-merror:0.138807+0.00117729	test-merror:0.147397+0.00109831
[2]	train-merror:0.133008+0.000985039	test-merror:0.142178+0.000551362
[3]	train-merror:0.130113+0.000817862	test-merror:0.139681+0.000418621
[4]	train-merror:0.12797+0.000792264	test-merror:0.137416+0.000719656
[5]	train-merror:0.125854+0.00120353	test-merror:0.13547+0.00124746
[6]	train-merror:0.12435+0.000748858	test-merror:0.134356+0.00113642
[7]	train-merror:0.122165+0.00102357	test-merror:0.132604+0.00114083
[8]	train-merror:0.120611+0.00101242	test-merror:0.131589+0.000995281
[9]	train-merror:0.119224+0.000837781	test-merror:0.130505+0.000789943
[10]	train-merror:0.117642+0.000861291	test-merror:0.129377+0.000712599
[11]	train-merror:0.11601+0.00089666	test-merror:0.128229+0.000883901
[12]	train-merror:0.114449+0.000721418	test-merror:0.127221+0.000655101
[13]	train-merror:0.113165+0.000580575	test-merror:0.126385+0.000459784
[14]	train-merror:

In [196]:
tuning_params = {'max_depth':[13, 14]}
model_params = {'n_estimators': 631, 'gamma':0.6,}
gridsearch(train_x, train_y, 'gs_maxDepth_minChildWeight_263_088_round2_.csv', tuning_params, model_params,
           cores=1, xgbthreads=-1)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] max_depth=10 ....................................................
[CV] ........................... max_depth=10, score=0.906232 -18.0min
[CV] max_depth=10 ....................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 18.0min remaining:    0.0s


[CV] ........................... max_depth=10, score=0.909841 -17.9min
[CV] max_depth=10 ....................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 35.9min remaining:    0.0s


[CV] ........................... max_depth=10, score=0.911057 -18.0min
[CV] max_depth=10 ....................................................


...

[CV] ........................... max_depth=11, score=0.906969 -19.7min
[CV] max_depth=11 ....................................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 168.5min remaining:    0.0s


[CV] ........................... max_depth=11, score=0.909077 -19.6min
[CV] max_depth=12 ....................................................
[CV] ........................... max_depth=12, score=0.907201 -21.4min
[CV] max_depth=12 ....................................................
[CV] ........................... max_depth=12, score=0.911285 -21.3min
[CV] max_depth=12 ....................................................
[CV] ........................... max_depth=12, score=0.912653 -21.3min
[CV] max_depth=12 ....................................................
[CV] ........................... max_depth=12, score=0.907558 -21.4min
[CV] max_depth=12 ....................................................
[CV] ........................... max_depth=12, score=0.909723 -21.3min


[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed: 294.8min finished


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0.6, learning_rate=0.1, max_delta_step=0, max_depth=12,
       min_child_weight=1, missing=None, n_estimators=631, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8) 0.9096842121267409
Done in 321.13327064116794 minutes (used cores: 1)
